<a href="https://colab.research.google.com/github/syoooooung/capstone_design/blob/main/NER/BERT_csv_create.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q groq
!pip install -U accelerate bitsandbytes datasets evaluate
!pip install -U peft transformers trl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.5/103.5 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 1.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusol

In [ ]:
# For Google Colab settings
from google.colab import userdata, drive

# This will prompt for authorization
drive.mount('/content/drive')

# Set the working directory
%cd '/content/drive/MyDrive/RE/llama3RE'

# # For Hugging Face Hub setting
# from huggingface_hub import login

# # Upload the HuggingFace token (should have WRITE access) from Colab secrets
# HF = userdata.get('HF')

# # This is needed to upload the model to HuggingFace
# login(token=HF,add_to_git_credential=True)

# Create a path variable for the data folder
data_path = '/content/drive/MyDrive/RE/llama3RE/datas/'

# Full fine-tuning dataset
sft_dataset_file = f'{data_path}sft_train_data.json'

# Data collected from the the mini-test
mini_data_path = f'{data_path}mini_data.json'

# Test data containing all three outputs
all_tests_data = f'{data_path}all_tests.json'

# The adjusted training dataset
train_data_path = f'{data_path}sft_train_data.json'

# Create a path variable for the SFT model to be saved locally
sft_model_path = '/content/drive/MyDrive/RE/llama3RE/Llama3_RE/'

Mounted at /content/drive
/content/drive/MyDrive/RE/llama3RE


In [ ]:
# For Hugging Face Hub setting
from huggingface_hub import login

# Upload the HuggingFace token (should have WRITE access) from Colab secrets
HF = userdata.get('HF')

# This is needed to upload the model to HuggingFace
login(token=HF,add_to_git_credential=True)

Token is valid (permission: write).
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
from datasets import load_dataset

# Load the dataset
dataset = load_dataset("databricks/databricks-dolly-15k")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/15011 [00:00<?, ? examples/s]

In [ ]:
import json
import csv
import time
from tqdm import tqdm
# Choose the desired category from the dataset
ie_category = [e for e in dataset["train"] if e["category"]=="information_extraction"]

# Retain only the context from each instance
ie_context = [e["context"] for e in ie_category]

# Split the text into sentences (at the period) and keep the first sentence
reduced_context = [text.split('.')[0] + '.' for text in ie_context]

# Retain sequences of specified lengths only (use character length)
sampler = [e for e in reduced_context if 30 < len(e) < 170]

json_path = "/content/dataset.json"
csv_output_path = "/content/output.csv"

# JSON 파일 로드
with open(json_path, 'r') as file:
    data = json.load(file)

# Query 부분만 추출하는 함수
def extract_query(input_text):
    query_start = input_text.find("Query:") + len("Query:")
    query_end = input_text.find("Entity:")
    query = input_text[query_start:query_end].strip()
    return query

In [ ]:
system_message = """
You are given a JSON object with 'input' and 'output' fields. Your task is to convert this information into a CSV format.
For each 'input', extract the Query and tokenize it. Then, tag each token (word) with the corresponding POS and entity tag based on the 'output' field.

The CSV format should follow this structure:
Sentence #,Word,POS,Tag

Each new sentence should begin with "Sentence: #" where # is the sentence number. The words in each sentence should be listed in the following format:
,Word,POS,Tag
The Tags should be one of the following: {B-core, I-core, B-etc, I-etc, O}.
POS should be one of the following: {CC, CD, DT, EX, FW, IN, JJ, JJR, JJS, LS, MD, NN, NNS, NNP, NNPS, PDT, POS, PRP, PRP$, RB, RBR, RBS, RP, SYM, TO, UH, VB, VBD, VBG, VBN, VBP, VBZ, WDT, WP, WP$, WRB, X}.
Ensure that the words are properly tokenized, and apply the correct POS and entity tag to each token based on the provided information.
Do not answer anything else.

Example:
Me:
Sentence: Were Scott Derrickson and Ed Wood of the nationality?\nEntity tags: Scott Derrickson (etc), Ed Wood (etc), nationality (core)
You:
Sentence: 1\n,Were,VBD,O\n,Scott,NNP,B-etc\n,Derrickson,NNP,I-etc\n,and,CC,O\n,Ed,NNP,B-etc\n,Wood,NNP,I-etc\n,of,IN,O\n,the,DT,O\n,nationality,NN,B-core\n,?,?,O
"""

In [ ]:
messages = [
    [
        {"role": "system", "content": f"{system_message}"},
        {"role": "user", "content": f"Sentence: {extract_query(item['input'])}\nEntity tags: {item['output']}"}
    ]
    for item in data
]

In [ ]:
import os
from groq import Groq

gclient = Groq(
    api_key="Groq_api_key",
)

In [ ]:

def process_data(prompt):
    """LLM에 메시지를 보내고 응답을 받습니다."""
    chat_completion = gclient.chat.completions.create(
        messages=prompt,
        model="llama3-70b-8192",
        temperature=0.5,
        max_tokens=128,
        top_p=1,
        stop=None,
        stream=False
    )
    return chat_completion.choices[0].message.content


def send_messages(messages):
    """메시지를 배치 단위로 처리하고 잠시 대기합니다."""
    batch_size = 10
    answers = []

    for i in tqdm(range(0, len(messages), batch_size)):
        batch = messages[i:i+batch_size]

        for message in batch:
            output = process_data(message)
            answers.append(output)

        if i + batch_size < len(messages):
            time.sleep(10)  # 다음 배치 처리 전에 10초 대기

    return answers


In [ ]:
# 메시지 처리
answers = send_messages(messages)

100%|██████████| 7/7 [05:38<00:00, 48.31s/it]


In [ ]:
answers

['Sentence: 1\n,Were,VBD,O\n,Scott,NNP,B-etc\n,Derrickson,NNP,I-etc\n,and,CC,O\n,Ed,NNP,B-etc\n,Wood,NNP,I-etc\n,of,IN,O\n,the,DT,O\n,same,JJ,O\n,nationality,NN,B-core\n,?,?,O',
 'Sentence: 1\n,What,WDT,B-etc\n,government,NNS,B-etc\n,position,NN,B-etc\n,was,VBD,O\n,held,VBN,O\n,by,IN,O\n,the,DT,O\n,woman,NN,B-core\n,who,WDT,O\n,portrayed,VBD,O\n,Corliss,NNP,B-etc\n,Archer,NNP,I-etc\n,in,IN,O\n,the,DT,O\n,film,NN,B-etc\n,Kiss,NNP,I-etc\n,and,CC,O\n,Tell,',
 'Sentence: 1\n,What,WDT,B-etc\n,science,NN,B-core\n,fantasy,NN,I-core\n,young,JJ,I-core\n,adult,JJ,I-core\n,series,NN,I-core\n,,SYM,O\n,told,VBD,O\n,in,IN,O\n,first,JJ,B-etc\n,person,NN,I-etc\n,,SYM,O\n,has,VBZ,O\n,a,DT,O\n,set,NN,O\n,of,IN,O\n,companion,NN,B-etc\n,books,NNS,I-etc\n,narrating,V',
 'Sentence: 1\n,Are,VBP,O\n,the,DT,O\n,Laleli,NNS,B-etc\n,Mosque,NNP,I-etc\n,and,CC,O\n,Esma,NNP,B-etc\n,Sultan,NNP,I-etc\n,Mansion,NNP,I-etc\n,located,VBN,O\n,in,IN,O\n,the,DT,O\n,same,JJ,B-core\n,neighborhood,NN,I-core\n,?,=?,O\n,?,=?,O',


In [ ]:
import re

In [ ]:
with open(csv_output_path, 'w', newline='', encoding='utf-8') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(["Sentence #", "Word", "POS", "Tag"])

    sentence_count = 1

    for answer in answers:
        # LLM의 응답에서 불필요한 텍스트 제거
        clean_answer = re.sub(r"Here is the (output|CSV format)[^:]*:\n\n", "", answer)

        # 응답을 줄 단위로 나누기
        lines = clean_answer.strip().split('\n')
        csvwriter.writerow([f"Sentence: {sentence_count}"])
        for line in lines:
            # 비어 있는 줄 무시
            if not line.strip():
                continue
            # 각 라인을 CSV 형식으로 처리
            columns = line.strip().split(',')
            if len(columns) >= 3:
                csvwriter.writerow([""] + columns[1:])  # 첫 번째 열을 빈 값으로 유지
        sentence_count += 1